In [3]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from functools import partial

import seaborn as sns

import pickle

In [ ]:
df = pd.read_csv('#EXPORT OF DATASET GOES HERE', low_memory=False)
df.shape


In [ ]:
X = df.drop(columns=['home_team_win', 'game_id'])
y = df.home_team_win

In [ ]:
def train_xgb(params):
    for i in range(10):
        params['max_depth'] = int(params['max_depth'])
        gbm = xgb.XGBClassifier(**params, num_boost_round=100, random_state=13, n_jobs=-1)
        model = gbm.fit(X_train, y_train,
                        verbose=False,
                        eval_set = [[X_train, y_train],
                                  [X_valid, y_valid]],
                        early_stopping_rounds=15,
                        callbacks=[XGBCallback])
        xgb_test_preds = model.predict(X_valid)
        proba = model.predict_proba(X_valid)[:,1]

        #penalize low discrimination
        score = roc_auc_score(y_valid, xgb_test_preds)
        if max(proba)<.7 and min(proba)>.3: score -= 0.05

        tune.track.log(mean_accuracy=score, done=True)



In [ ]:
num_threads = 6
config = {
    "verbosity": 0,
    "num_threads": num_threads,
    "objective": "binary:logistic",
    "eval_metric": ["auc", "logloss"],
}

from ray.tune.schedulers import ASHAScheduler
space = {
    "max_depth": (10, 20),
    "min_child_weight": (2, 10),
    "learning_rate": (0.01,0.1),
    "subsample":(0.1,1),
    "colsample_bytree":(0.1,1),
    "reg_alpha": (1e-2, 1e2),
    "gamma": (1e-8, 1.0),
}

algo = BayesOptSearch(
    space,
    max_concurrent=2,
    metric="mean_accuracy",
    mode="max",
    utility_kwargs={
            "kind": "ucb",
            "kappa": 2.5,
            "xi": 0.0,
        },
)

results = tune.run(
    train_xgb,
    num_samples=500,
    search_alg=algo,
    scheduler=ASHAScheduler(metric="eval-logloss", mode="min"),
)